In [62]:
import sys, re
from molvs import Standardizer
from rdkit import Chem
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import errorcode
import operator
from rdkit.Chem.SaltRemover import SaltRemover
import sys
import re
from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from optparse import OptionParser
import pandas as pd
import numpy as np
from rdkit.Chem.Descriptors import MolWt, HeavyAtomCount,MolLogP, RingCount, NumAromaticRings, NumHDonors, NumHAcceptors
from rdkit.Chem.Descriptors import NumAromaticRings, qed
from rdkit.Chem.QED import QEDproperties
from rdkit.Chem import rdMMPA
from indexing_call_as_function import cansmirk,add_context_to_smirks,index_from_smiles_re,gen_queries,gen_queries_for_H_transform,add_context_to_lhs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from molvs import standardize_smiles, Standardizer
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem.SaltRemover import SaltRemover

# Download the data and prepare

In [63]:
df = pd.read_csv('../MMP_files/CDD_18_April_2022.csv')

C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1,6,11,12,22,24,25,26,27,30,31,32,33,34,41,42,43,44,45,46,47,48,49,50,51,56,57,60,61,62,63,64,65,66,67,68,69,70,71,72,75,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
df.columns

Index(['Molecule Name', 'CDD Number', 'CXSMILES (CDD Compatible)', 'log P',
       'log D', 'Topological polar surface area',
       'ERK5 Biochem IC50: Project Name',
       'ERK5 Biochem IC50: ERK5 Biochem IC50 (nM)',
       'ERK5 Biochem IC50: Avg IC50 (nM)',
       'ERK5 Biochem IC50: Avg IC50 (nM) Standard Deviation (±)',
       'ERK5 Biochem IC50: Avg IC50 (nM) Count', 'hERG: IC50 (uM)',
       'Kinetic Solubility (FaSSIF): Kinetic Solubility (uM)',
       'Measured logD: logD', 'Pharmacokinetic: Species',
       'Pharmacokinetic: Cmax (ng/mL)', 'Pharmacokinetic: Tmax (h)',
       'Pharmacokinetic: T1/2 (h)', 'Pharmacokinetic: Vdss (L/kg)',
       'Pharmacokinetic: Cl (mL/min/kg)',
       'Pharmacokinetic: AUC 0-last (ng*h/mL)',
       'Pharmacokinetic: Bioavailability (%)', 'Plasma Stability: Species',
       'Plasma Stability: Parent remaining at 120 min (%)',
       'Plasma Stability: T1/2', 'ERK5 MEF2 IC50 Paraza: Project Name',
       'ERK5 MEF2 IC50 Paraza: ERK5 Mef2 IC50 P

In [65]:
df[:10]

,Molecule Name,CDD Number,CXSMILES (CDD Compatible),log P,log D,Topological polar surface area,ERK5 Biochem IC50: Project Name,ERK5 Biochem IC50: ERK5 Biochem IC50 (nM),ERK5 Biochem IC50: Avg IC50 (nM),ERK5 Biochem IC50: Avg IC50 (nM) Standard Deviation (±),...,PC9 (WT) CTG: IC50 (uM),PC9 (WT) CTG: IC95 (uM),PC9 (WT) CTG: Average IC50 (nM),pERK PC9 (WT) HTRF (4 hour): IC50 (uM),pERK PC9 (WT) HTRF (4 hour): IC95 (uM),pERK PC9 (WT) HTRF (4 hour): Average IC50 (nM),WT KRas/CRAF GMPPNP HTRF: IC50 (uM),WT KRas/CRAF GMPPNP HTRF: Average IC50 (nM),WT KRas *GTP HTRF: IC50 (uM),WT KRas *GTP HTRF: Average IC50 (nM)
0,IK-0013784,NaN,CNS(=O)(=O)NC1=CC(F)=C(CN2CC3=C(OC2=O)C=C(OC2=...,2.47,2.47,109.86,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IK-0013783,NaN,NS(=O)(=O)NC1=CC(F)=C(CN2CC3=C(OC2=O)C=C(OC2=N...,2.25,2.25,123.85,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IK-0013782,NaN,CNS(=O)(=O)NC1=NC=CC(CN2[C@@H](C)C3=C(OC2=O)C=...,2.41,2.40,122.75,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IK-0013781,NaN,CNS(=O)(=O)NC1=NC=CC(CN2[C@@H](C)C3=C(OC2=O)C(...,2.41,2.40,122.75,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IK-0013780,NaN,CNS(=O)(=O)NC1=C(F)C(=CC=C1)[C@H](C)N1[C@@H](C...,3.31,3.30,109.86,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,IK-0013779,NaN,CNS(=O)(=O)NC1=C(F)C(=CC=C1)[C@H](C)N1[C@H](C)...,3.31,3.30,109.86,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,IK-0013759,NaN,CNS(=O)(=O)NC1=NC=CC(CN2CC3=C(OC2=O)C=C(OC2=NC...,1.85,1.84,122.75,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,IK-0013759,NaN,CNS(=O)(=O)NC1=NC=CC(CN2CC3=C(OC2=O)C=C(OC2=NC...,1.85,1.84,122.75,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,IK-0013759,NaN,CNS(=O)(=O)NC1=NC=CC(CN2CC3=C(OC2=O)C=C(OC2=NC...,1.85,1.84,122.75,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,IK-0013759,NaN,CNS(=O)(=O)NC1=NC=CC(CN2CC3=C(OC2=O)C=C(OC2=NC...,1.85,1.84,122.75,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df.columns

Index(['Molecule Name', 'CDD Number', 'CXSMILES (CDD Compatible)', 'log P',
       'log D', 'Topological polar surface area',
       'ERK5 Biochem IC50: Project Name',
       'ERK5 Biochem IC50: ERK5 Biochem IC50 (nM)',
       'ERK5 Biochem IC50: Avg IC50 (nM)',
       'ERK5 Biochem IC50: Avg IC50 (nM) Standard Deviation (±)',
       'ERK5 Biochem IC50: Avg IC50 (nM) Count', 'hERG: IC50 (uM)',
       'Kinetic Solubility (FaSSIF): Kinetic Solubility (uM)',
       'Measured logD: logD', 'Pharmacokinetic: Species',
       'Pharmacokinetic: Cmax (ng/mL)', 'Pharmacokinetic: Tmax (h)',
       'Pharmacokinetic: T1/2 (h)', 'Pharmacokinetic: Vdss (L/kg)',
       'Pharmacokinetic: Cl (mL/min/kg)',
       'Pharmacokinetic: AUC 0-last (ng*h/mL)',
       'Pharmacokinetic: Bioavailability (%)', 'Plasma Stability: Species',
       'Plasma Stability: Parent remaining at 120 min (%)',
       'Plasma Stability: T1/2', 'ERK5 MEF2 IC50 Paraza: Project Name',
       'ERK5 MEF2 IC50 Paraza: ERK5 Mef2 IC50 P

In [67]:
# pERK AsPC-1 (G12D) HTRF (4 hour): IC50 (uM)

In [68]:
df['pERK AsPC-1 (G12D) HTRF (4 hour): IC50 (uM)'].dropna()

16         0.476
17        > 20.0
18        > 20.0
19        > 20.0
20      2.82E-03
          ...   
6480     691E-06
6481     560E-06
6482     416E-06
6483     416E-06
7621      > 20.0
Name: pERK AsPC-1 (G12D) HTRF (4 hour): IC50 (uM), Length: 461, dtype: object

In [69]:
cols = ['ERK5 Biochem IC50: Avg IC50 (nM)','hERG: IC50 (uM)',
        'Kinetic Solubility (FaSSIF): Kinetic Solubility (uM)',
       'Measured logD: logD','Pharmacokinetic: Cmax (ng/mL)','Pharmacokinetic: Tmax (h)',
       'Pharmacokinetic: T1/2 (h)', 'Pharmacokinetic: Vdss (L/kg)',
       'Pharmacokinetic: Cl (mL/min/kg)',
       'Pharmacokinetic: Bioavailability (%)','Plasma Stability: T1/2',
       'ERK5 MEF2 IC50 Paraza: Avg IC50 (nM)','Caco2 w/BSA: Papp (A to B) (10-6 cm/s)',
       'Caco2 w/BSA: Paap (B to A) (10-6 cm/s)', 'Caco2 w/BSA: Efflux Ratio','CRaf/MEK SPR: KD (M)',
       'FBS Binding (Equilibrium Dialysis): Unbound (%)',
       'FBS Binding (Equilibrium Dialysis): Bound (%)',
       'FBS Binding (Equilibrium Dialysis): Remaining (%)','Hepatocyte stability: T1/2 (min)',
        'Hepatocyte stability: ER (Ratio)',
       'Hepatocyte stability: Clint (mL/min/kg)','pERK AsPC-1 (G12D) HTRF (4 hour): IC50 (uM)',
       'pERK AsPC-1 (G12D) HTRF (4 hour): Average IC50 (nM)','pERK AsPC-1 (G12D) ICW (4 hour): IC50 (uM)',
       'pERK AsPC-1 (G12D) ICW (4 hour): IC95 (uM)',
       'pERK AsPC-1 (G12D) ICW (4 hour): Average IC50 (nM)','Plasma Protein Binding (dialysis): Unbound (%)',
       'Plasma Protein Binding (dialysis): Bound (%)','Plasma Protein Binding ucentri: Unbound (%)',
       'Plasma Protein Binding ucentri: Recovery (%)',
       'AsPC-1 (G12D) CTG: % inhibition (%)',
       'AsPC-1 (G12D) CTG: Average IC50 (nM)',
       'G12C KRas *GTP HTRF: IC50 (uM)',
       'G12C KRas *GTP HTRF: Average IC50 (nM)','G12C KRas *GTP HTRF: Selectivity (G12C/WT)',
       'G12D KRas/CRAF GMPPNP HTRF: IC50 (uM)',
       'G12D KRas/CRAF GMPPNP HTRF: Average IC50 (nM)',
       'G12D KRas/CRAF GMPPNP HTRF: Selectivity (G12D/WT)',
       'G12D KRas *GTP HTRF: IC50 (uM)',
       'G12D KRas *GTP HTRF: Average IC50 (nM)',
       'G12V KRas *GTP HTRF: IC50 (uM)',
       'G12V KRas *GTP HTRF: Average IC50 (nM)',
       'G12V KRas *GTP HTRF: Selectivity (G12V/WT)',
       'MDCK w/BSA & PGP Inhibitor: Papp (A to B) (10-6 cm/s)',
       'MDCK w/BSA & PGP Inhibitor: Papp (B to A) (10-6 cm/s)',
       'MDCK w/BSA & PGP Inhibitor: Efflux Ratio',
       'MDCK w/BSA & PGP Inhibitor: Recovery (A to B) (%)',
       'MDCK w/BSA & PGP Inhibitor: Recovery (B to A) (%)',
       'Microsomal Stability: T1/2 (min)',
       'Microsomal Stability: ER (Ratio)',
       'Microsomal Stability: Clint (mL/min/kg)', 'PAMPA_egg: Pe',
       'PAMPA_egg: Recovery (%)', 'PC9 (WT) CTG: IC50 (uM)',
       'PC9 (WT) CTG: IC95 (uM)', 'PC9 (WT) CTG: Average IC50 (nM)',
       'pERK PC9 (WT) HTRF (4 hour): IC50 (uM)',
       'pERK PC9 (WT) HTRF (4 hour): IC95 (uM)',
       'pERK PC9 (WT) HTRF (4 hour): Average IC50 (nM)',
       'WT KRas/CRAF GMPPNP HTRF: IC50 (uM)',
       'WT KRas/CRAF GMPPNP HTRF: Average IC50 (nM)',
       'WT KRas *GTP HTRF: Average IC50 (nM)','log P',
       'log D', 'Topological polar surface area']

In [70]:
# for x in cols:
#     table = "_".join([y for y in re.split('[-\%\*\/\:\(\)& ]',x)])
#     print("<li>"+table.lower()+"</li>")

In [71]:
len(cols)

66

In [72]:
cols[63:64]

['log P']

In [73]:
df.rename(columns = {'Plasma Protein Binding (ultracentrifugation): Unbound (%)' : 'Plasma Protein Binding ucentri: Unbound (%)', 
                     'Plasma Protein Binding (ultracentrifugation): Recovery (%)' : 'Plasma Protein Binding ucentri: Recovery (%)'}, inplace = True)

In [74]:
df['WT KRas *GTP HTRF: IC50 (uM)'].dropna()

49          0.149
51           3.59
53         > 10.0
55         > 10.0
57         > 10.0
          ...    
5969        0.175
6384        0.282
6427    < 169E-06
6428       > 10.0
7621       > 10.0
Name: WT KRas *GTP HTRF: IC50 (uM), Length: 353, dtype: object

In [75]:
config = {
  'user': 'sandeep',
  'password': 'sql',
  'host': 'localhost',
  'database': 'mmpdb',
  'raise_on_warnings': True,
}

link = mysql.connector.connect(**config)

In [76]:
cnx= mysql.connector.connect(**config)
cur = cnx.cursor()

# Drop all tables and start fresh

In [77]:
# i = 1
# for x in cols:
    
# #     print("The columns number = {}".format(i))
#     i = i + 1

#     table = "_".join([y for y in re.split('[-\%\*\/\:\(\)& ]',x)])
# #     print(table)
#     try:
#         sql = 'DROP TABLE IF EXISTS {} CASCADE;'.format(table)
#         cur.execute(sql)
#     except:
#         None
    
    
    
# sql = 'DROP TABLE IF EXISTS transform CASCADE;'
# print(sql)
# cur.execute(sql)
# sql = 'DROP TABLE IF EXISTS transform_left CASCADE;'
# print(sql)
# cur.execute(sql)
# sql = 'DROP TABLE IF EXISTS context CASCADE;'
# print(sql)
# cur.execute(sql)
# sql = 'DROP TABLE IF EXISTS context_type CASCADE;'
# print(sql)
# cur.execute(sql)
# sql = 'DROP TABLE IF EXISTS mmp CASCADE;'
# print(sql)
# cur.execute(sql)

# cur.execute('DROP TABLE IF EXISTS ikenacomps')


In [78]:
# sql = 'CREATE TABLE if not exists ikenacomps (id integer not null auto_increment, '\
#       'Molecule_id varchar(128) unique, '\
#       'CXCSmiles varchar(512), '\
#      'primary key(`id`)) ENGINE=InnoDB'
# print(sql)
# cur.execute(sql)

In [79]:
# id_s = []
# for _, x in df.iterrows():
    
#     if (x['Molecule Name'] not in id_s):
        
#         sql = 'INSERT INTO ikenacomps (Molecule_id, CXCSmiles ) ' \
#             ' VALUES (%s, %s)'
#         cur.execute(sql,(x['Molecule Name'],x['CXSMILES (CDD Compatible)']
#                         ))
#         id_s.append(x['Molecule Name'])

In [80]:
def calc_aggregation(group,name):
    name_mean = name+"_mean";
    name_count = name+"_count";
    name_std = name+"_std";
    name_data = name+"_data"
#     print([re.sub('[><= ]+ ','',str(x)) for x in group[name]])
    out = [float(re.sub('[><= ]+ ','',str(x))) for x in group[name]]
    group['mean'] = "{:0.2f}".format(np.mean(out))
    group['count'] = np.size(out)
#     group['std_dev']= np.std(out,ddof=1)
    group['data'] = ";".join([str(x) for x in group[name]])
    return group 

In [81]:
df['Pharmacokinetic: AUC 0-last (ng*h/mL)']

0        6671.0
1       47282.0
2        6227.0
3        6120.0
4        5416.0
         ...   
8392     2681.0
8393    10892.0
8394     2750.0
8395      784.0
8396        NaN
Name: Pharmacokinetic: AUC 0-last (ng*h/mL), Length: 8397, dtype: float64

In [82]:
i = 1
for x in cols:
    
    print("The columns number = {}".format(i))
    i = i + 1

    table = "_".join([y for y in re.split('[-\%\*\/\:\(\)& ]',x)])
    sql = 'DROP TABLE IF EXISTS {} CASCADE;'.format(table)
    cur.execute(sql)
    sql = 'CREATE TABLE if not exists {} (id integer not null auto_increment, Molecule_id varchar(128) unique, '\
     '{} varchar(20), mean double, '\
     'count varchar(200), data text, ' \
     'mol_id integer,'\
     'constraint foreign key (mol_id) '\
     'references ikenacomps(id) on delete cascade on update cascade, '\
     'primary key(id)) ENGINE=InnoDB'.format(table,table)
    print(sql)
    cur.execute(sql)
  
    df1 = df[['Molecule Name','CXSMILES (CDD Compatible)',x]]
    df1.dropna(subset=[x],inplace=True)
    df1 = pd.DataFrame(df1.groupby("Molecule Name").apply(calc_aggregation, x))
    
    id_s = []
    
    for _,y in df1.iterrows():
        
        if(y['Molecule Name'] not in id_s):
 
            sql = 'insert into {} (Molecule_id, {}, mean, count, data) '\
                  'values (%s,%s,%s,%s,%s)'.format(table,table)

            cur.execute(sql,(y['Molecule Name'],y[x],y['mean'],y['count'],y['data']))

            id_s.append(y['Molecule Name'])
            
    sql = 'update {} set mol_id = (select ikenacomps.id from ikenacomps ' \
          'where ikenacomps.Molecule_id = {}.Molecule_id)'.format(table,table)
    
    cur.execute(sql,())

The columns number = 1
CREATE TABLE if not exists ERK5_Biochem_IC50__Avg_IC50__nM_ (id integer not null auto_increment, Molecule_id varchar(128) unique, ERK5_Biochem_IC50__Avg_IC50__nM_ varchar(20), mean double, count varchar(200), data text, mol_id integer,constraint foreign key (mol_id) references ikenacomps(id) on delete cascade on update cascade, primary key(id)) ENGINE=InnoDB


C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The columns number = 2
CREATE TABLE if not exists hERG__IC50__uM_ (id integer not null auto_increment, Molecule_id varchar(128) unique, hERG__IC50__uM_ varchar(20), mean double, count varchar(200), data text, mol_id integer,constraint foreign key (mol_id) references ikenacomps(id) on delete cascade on update cascade, primary key(id)) ENGINE=InnoDB


C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The columns number = 3
CREATE TABLE if not exists Kinetic_Solubility__FaSSIF___Kinetic_Solubility__uM_ (id integer not null auto_increment, Molecule_id varchar(128) unique, Kinetic_Solubility__FaSSIF___Kinetic_Solubility__uM_ varchar(20), mean double, count varchar(200), data text, mol_id integer,constraint foreign key (mol_id) references ikenacomps(id) on delete cascade on update cascade, primary key(id)) ENGINE=InnoDB


...

C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The columns number = 65
CREATE TABLE if not exists log_D (id integer not null auto_increment, Molecule_id varchar(128) unique, log_D varchar(20), mean double, count varchar(200), data text, mol_id integer,constraint foreign key (mol_id) references ikenacomps(id) on delete cascade on update cascade, primary key(id)) ENGINE=InnoDB


C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The columns number = 66
CREATE TABLE if not exists Topological_polar_surface_area (id integer not null auto_increment, Molecule_id varchar(128) unique, Topological_polar_surface_area varchar(20), mean double, count varchar(200), data text, mol_id integer,constraint foreign key (mol_id) references ikenacomps(id) on delete cascade on update cascade, primary key(id)) ENGINE=InnoDB


C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Standardization

In [83]:
s= Standardizer()
remover = SaltRemover(defnData="[Cl,Br,Na,K,F,I,Mg,Ca]")
df['smiles'] = df['CXSMILES (CDD Compatible)'].apply(lambda x: Chem.MolToSmiles(s.standardize(remover.StripMol(Chem.MolFromSmiles(x)))))

In [84]:
df['HeavyAtomCount'] = df['smiles'].apply(lambda x: HeavyAtomCount(Chem.MolFromSmiles(x)))

In [85]:
df.drop_duplicates(subset=['smiles'],inplace=True)

In [86]:
df['frags'] = df['smiles'].apply(lambda x: rdMMPA.FragmentMol(Chem.MolFromSmiles(x), pattern="[#6+0;!$(*=,#[!#6])]!@!=!#[*]", resultsAsMols=False))

In [87]:
index = index_from_smiles_re(df)

In [88]:
def mol_with_atom_index(mol):
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return mol

In [89]:
def get_context_type(context):

    mol = Chem.MolFromSmiles(context)
    mol = mol_with_atom_index(mol)
    
    atomindex = []
    atoms = {}
    neigh = {}
    lv1_keys = []
    
    for atom in mol.GetAtoms():
        atoms[atom.GetIdx()] = atom
        neigh[atom.GetIdx()] = atom.GetNeighbors()

        if(atom.GetMass()==0):
            atomindex.append(atom.GetIdx())
            lv1_keys.append(atom.GetIdx())
    
    str_ = []
    for keys in lv1_keys:
        for values in neigh[keys]:
            if(values.GetIsAromatic()):
                str_.append("a")
            else:
                str_.append("A")
                
    return "".join(str_)

In [90]:
import time
start_time = time.time()

transform_lev0 = []
context_type = []
id_left = []
id_right = []
contextcont = []

for key in index:
    total = len(index[key])
    if(total == 1):  # Meaning that this context does not have any Mathed Pairs therefore exit the for loop
        continue
    for xa in range(total-1):
        for xb in range(xa+1,total):
            if(xa != xb):
                id_a,core_a = index[key][xa].split(";t")
                id_b,core_b = index[key][xb].split(";t")
                if(id_a != id_b):
                    if(core_a != core_b):
                        smirks,context = cansmirk(core_a,core_b,key)
                        id_a = int(id_a)
                        id_b = int(id_b)
                        context_type.append(get_context_type(context))
                        transform_lev0.append(smirks)
                        contextcont.append(context)
                        id_left.append(df.loc[id_a]['Molecule Name'])
                        id_right.append(df.loc[id_b]['Molecule Name'])
                        
end_time = time.time()
print("total time taken this loop: ", end_time - start_time)

total time taken this loop:  61.03091359138489


In [91]:
df_mmp_lev0 = pd.DataFrame({'Reference_id' :id_left,
                           'Query_id' :id_right,
                           'transform' : transform_lev0,
                           'Ref_Context' :context_type,
                           'Query_Context' :context_type,
                           'Context': contextcont}
                          )

In [92]:
df_mmp_lev0.shape

(41356, 6)

# Create tables for transform, lhs, context and mmp_master_table

In [93]:
# sql = 'DROP TABLE IF EXISTS transform CASCADE;'
# print(sql)
# cur.execute(sql)
# sql = 'DROP TABLE IF EXISTS transform_left CASCADE;'
# print(sql)
# cur.execute(sql)
# sql = 'DROP TABLE IF EXISTS context CASCADE;'
# print(sql)
# cur.execute(sql)
# sql = 'DROP TABLE IF EXISTS context_type CASCADE;'
# print(sql)
# cur.execute(sql)
# sql = 'DROP TABLE IF EXISTS mmp CASCADE;'
# print(sql)
# cur.execute(sql)

In [94]:
sql = 'CREATE TABLE if not exists transform (id integer not null auto_increment,'\
      'transform varchar(250) binary unique,' \
     'primary key(id)) ENGINE=InnoDB'
print(sql)
cur.execute(sql)
sql = 'CREATE TABLE if not exists transform_left (id integer not null auto_increment,'\
      'transform_left varchar(250) binary unique,' \
     'primary key(id)) ENGINE=InnoDB'
print(sql)
cur.execute(sql)
sql = 'CREATE TABLE if not exists context (id integer not null auto_increment,'\
      'context varchar(250) binary unique,' \
     'primary key(id)) ENGINE=InnoDB'
print(sql)
cur.execute(sql)
sql = 'CREATE TABLE if not exists context_type (id integer not null auto_increment,'\
      'context_type varchar(250) binary unique,' \
     'primary key(id)) ENGINE=InnoDB'
print(sql)
cur.execute(sql)
sql = 'create table if not exists mmp (id integer not null auto_increment, ref_id varchar(50), id_a integer, '\
      'query_id varchar(50), id_b integer, transform varchar(250), transform_id integer, ' \
      'lhs varchar(250), lhs_id integer, context_t varchar(10), context_t_id integer, ' \
      'context varchar(250), context_id integer, primary key(id)) ENGINE=InnoDB'
print(sql)
cur.execute(sql)

CREATE TABLE if not exists transform (id integer not null auto_increment,transform varchar(250) binary unique,primary key(id)) ENGINE=InnoDB
CREATE TABLE if not exists transform_left (id integer not null auto_increment,transform_left varchar(250) binary unique,primary key(id)) ENGINE=InnoDB
CREATE TABLE if not exists context (id integer not null auto_increment,context varchar(250) binary unique,primary key(id)) ENGINE=InnoDB
CREATE TABLE if not exists context_type (id integer not null auto_increment,context_type varchar(250) binary unique,primary key(id)) ENGINE=InnoDB
create table if not exists mmp (id integer not null auto_increment, ref_id varchar(50), id_a integer, query_id varchar(50), id_b integer, transform varchar(250), transform_id integer, lhs varchar(250), lhs_id integer, context_t varchar(10), context_t_id integer, context varchar(250), context_id integer, primary key(id)) ENGINE=InnoDB


In [95]:
transforms = []
one_side = []
context = []
context_t = []

for _,x in df_mmp_lev0.iterrows():
    
    trans_a,trans_b = x['transform'].split(">>")
    opposite_transform = trans_b+">>"+trans_a
    
    if(x['transform'] not in transforms):
        sql = 'INSERT INTO transform (transform) VALUES (%s);'
        cur.execute(sql, (x['transform'], ))
        transforms.append(x['transform'])
        
    if(opposite_transform not in transforms):
        sql = 'insert into transform (transform) values (%s);'
        cur.execute(sql,(opposite_transform, ))
        transforms.append(opposite_transform)
        
    if(trans_a not in one_side):
        sql = 'insert into transform_left (transform_left) values (%s);'
        cur.execute(sql,(trans_a, ))
        one_side.append(trans_a)
        
    if(trans_b not in one_side):
        sql = 'insert into transform_left (transform_left) values (%s);'
        cur.execute(sql,(trans_b, ))
        one_side.append(trans_b)
        
    if(x['Context'] not in context):
        sql = 'insert into context (context) values (%s);'
        cur.execute(sql,(x['Context'], ))
        context.append(x['Context'])  
        
    if(x['Ref_Context'] not in context_t):
        sql = 'insert into context_type (context_type) values (%s);'
        cur.execute(sql,(x['Ref_Context'], ))
        context_t.append(x['Ref_Context']) 
        
    sql = 'insert into mmp (ref_id,query_id,transform, lhs, context_t, context)' \
          'values (%s,%s,%s,%s,%s,%s);'
    cur.execute(sql,(x['Reference_id'],x['Query_id'], x['transform'], trans_a, x['Ref_Context'], x['Context']))
    
    
    sql = 'insert into mmp (ref_id,query_id,transform, lhs, context_t, context)' \
          'values (%s,%s,%s,%s,%s,%s);'
    cur.execute(sql,(x['Query_id'],x['Reference_id'], opposite_transform, trans_b, x['Query_Context'], x['Context']))

In [96]:
sql = 'update mmp set id_a = (select ikenacomps.id from ikenacomps ' \
      'where ikenacomps.Molecule_id = mmp.ref_id)'

cur.execute(sql,())

sql = 'update mmp set id_b = (select ikenacomps.id from ikenacomps ' \
      'where ikenacomps.Molecule_id = mmp.query_id)'

cur.execute(sql,())


sql = 'update mmp set transform_id = (select transform.id from transform ' \
      'where transform.transform = mmp.transform)'

cur.execute(sql,())


sql = 'update mmp set lhs_id = (select transform_left.id from transform_left ' \
      'where transform_left.transform_left = mmp.lhs)'

cur.execute(sql,())


sql = 'update mmp set context_t_id = (select context_type.id from context_type ' \
      'where context_type.context_type = mmp.context_t)'

cur.execute(sql,())


sql = 'update mmp set context_id = (select context.id from context ' \
      'where context.context = mmp.context)'

cur.execute(sql,())


sql = 'alter table mmp drop column ref_id,' \
      'drop query_id, drop transform, drop lhs, drop context_t, drop context, drop context_t_id '
cur.execute(sql,())  


In [97]:
cnx.commit()
cnx.close()